<a href="https://colab.research.google.com/github/IlyaZelinskiy/Wav2vec_CFT_SHIFT_LAB/blob/main/eng_to_ipa_learning_ver_for_recreate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install eng-to-ipa 
!pip install datasets==1.18.3
!pip install transformers==4.17.0

from datasets import load_dataset, load_metric
repo_name = "wav2vec2-base-timit-demo-google-colab"
timit = load_dataset("timit_asr")

In [2]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [3]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [4]:
timit = timit.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [10]:
!pip install eng-to-ipa 
import eng_to_ipa as p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
def extract_all_phonems(batch):
    all_text = p.convert(batch['text'])
    all_text = " ".join(all_text)
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [14]:
vocabs = timit.map(extract_all_phonems, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
#vocab_dict

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

46

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [5]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 9.3 MB/s 
     |████████████████████████████████| 2.2 MB 55.4 MB/s 


In [6]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [7]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Функция для прослушивания аудио

In [ ]:

import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=False, rate=16000)

a voice spoke nearathand   


In [ ]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 1680
    })
})

In [8]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(p.convert(batch["text"])).input_ids
    return batch

In [11]:
dataset = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

In [ ]:
max_input_length_in_sec = 5.0
dataset['train'] = dataset['train'].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    -Сопоставитель данных, который будет динамически дополнять полученные входные данные.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
            -Процессор, используемый для обработки данных.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            -
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided). - Дополнение к самой длинной последовательности в пакете (или без заполнения, если только одна
              последовательность, если она указана).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided. - Дополнить до максимальной длины, указанной аргументом :obj:`max_length`, или до
              максимальная приемлемая длина входных данных для модели, если этот аргумент не указан.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
import torchaudio
import torch
import numpy as np

from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
)
from torch import nn
model.lm_head = nn.Linear(in_features= 768, out_features= 46, bias = True)

In [ ]:
model.config.vocab_size = 46

In [21]:
from datasets import load_metric
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [22]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1709: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=repo_name,
    group_by_length=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    gradient_checkpointing=True,
    fp16=True,
    max_steps=15000,
    save_steps=1000,  #100,
    eval_steps=1000,
    logging_steps=100,
    learning_rate=3e-5,
    warmup_steps=2000,
    save_total_limit=3,
    load_best_model_at_end=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [ ]:
trainer.train('/content/drive/MyDrive/diploma/path')#после того как обучили часть модели, надо сохранить trainer строчкой
#ниже, потом здесь прописать путь откуда возьмется инфа о предыдущем обучении, и так повторяем пока не получим норм метрику
#потом юзаем save_pretrained и получаем fine tuning модель!
#на первом проходе трейнера путь не указываем!

Loading model from /content/drive/MyDrive/diploma/wer 0.51(0.27)).
The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4620
  Num Epochs = 105
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 15000


Step,Training Loss,Validation Loss,Wer
1000,0.644300,0.312975,0.290056
2000,0.521900,0.297662,0.276618
3000,0.462400,0.299377,0.276687
4000,0.409500,0.289382,0.273792
5000,0.358200,0.307106,0.273517
6000,0.353000,0.303311,0.270347


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8
Saving model checkpoint to wav2vec2-base-timit-demo-google-colab/checkpoint-1000
Configuration saved in wav2vec2-base-timit-demo-google-colab/checkpoint-1000/config.json
Model weights saved in wav2vec2-base-timit-demo-google-colab/checkpoint-1000/pytorch_model.bin
Feature extractor saved in wav2vec2-base-timit-demo-google-colab/checkpoint-1000/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8

Step,Training Loss,Validation Loss,Wer
1000,0.644300,0.312975,0.290056
2000,0.521900,0.297662,0.276618
3000,0.462400,0.299377,0.276687
4000,0.409500,0.289382,0.273792
5000,0.358200,0.307106,0.273517
6000,0.353000,0.303311,0.270347
7000,0.319900,0.315421,0.272276


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8
Saving model checkpoint to wav2vec2-base-timit-demo-google-colab/checkpoint-7000
Configuration saved in wav2vec2-base-timit-demo-google-colab/checkpoint-7000/config.json
Model weights saved in wav2vec2-base-timit-demo-google-colab/checkpoint-7000/pytorch_model.bin
Feature extractor saved in wav2vec2-base-timit-demo-google-colab/checkpoint-7000/preprocessor_config.json
Deleting older checkpoint [wav2vec2-base-timit-demo-google-colab/checkpoint-5000] due to args.save_total_limit


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model("path")
trainer.save_state()

Saving model checkpoint to path
Configuration saved in path/config.json
Model weights saved in path/pytorch_model.bin
Feature extractor saved in path/preprocessor_config.json


In [ ]:
import torch
torch.save(mod, '/content/drive/MyDrive/path')
import torch
m = torch.load('/content/drive/MyDrive/path')
m.save_pretrained('/content/drive/MyDrive/wav2vec_eng_to_ipa_wer(0.195)')

In [ ]:

import torchaudio
import torch
import numpy as np

from transformers import Wav2Vec2ForCTC
mod = Wav2Vec2ForCTC.from_pretrained('/content/drive/MyDrive/diploma/wav2vec_eng_to_ipa_wer(0.195)')
mod.to("cuda")

In [16]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [17]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = mod(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [19]:
results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

0ex [00:00, ?ex/s]

In [23]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.195


In [24]:
show_random_elements(results)

,pred_str,text
0,ʃi hæd jʊr dɑrk sut ɪn ˈgrisi wɑʃ ˈwɔtər ɔl jɪr,ʃi hæd jʊr dɑrk sut ɪn ˈgrisi wɑʃ ˈwɔtər ɔl jɪr
1,əˈlaʊ ˈliˌweɪ hir bət ˈræʃənəˌlaɪz ɔl ˈɛrərz,əˈlaʊ ˈliˌweɪ hir bət ˈræʃənəˌlaɪz ɔl ˈɛrərz
2,dɪˈsɛmbər ənd ˈʤænjəˌwɛri ər nis məns tɪ ˈspɛnd ɪn maɪ ˈæmi,dɪˈsɛmbər ənd ˈʤænjuˌɛri ər nis mənθs tɪ spɛnd ɪn maɪˈæmi
3,ˈwɛðər pruf kəˈlɑʃəz ər ˈvɛri ˈjusfəl ɪn siˈædəl,ˈwɛðərˌpruf gəˈlɑʃɪz ər ˈvɛri ˈjusfəl ɪn siˈætəl
4,jəŋ ˈʧɪldrən ʃʊd əˈvɔɪd ɪkˈspoʊʒər tɪ kənˈteɪʤəs dɪˈzizɪz,jəŋ ˈʧɪldrən ʃʊd əˈvɔɪd ɪkˈspoʊʒər tɪ kənˈteɪʤəs dɪˈzizɪz
5,doʊnt æsk mi tɪ ˈkɛri ən ˈɔɪli ræg laɪk ðət,doʊnt æsk mi tɪ ˈkɛri ən ˈɔɪli ræg laɪk ðət
6,ə ˈwʊmən mɛt ə ˈfeɪməs ˈɔθər æt ə ˈlɪtərrɛri ti,ə ˈwʊmən mɛt ə ˈfeɪməs ˈɔθər æt ə ˈlɪtərˌɛri ti
7,waɪ jɛl ər ˈwəriər wɪθ ˈsɪli ˈaɪtəmz,waɪ jɛl ər ˈwəri ˈoʊvər ˈsɪli ˈaɪtəmz
8,ʃi ˈreɪdiˌeɪtɪd wɔrmθ ənd gʊd ˈfɛloʊˌʃɪp,ʃi ˈreɪdiˌeɪtɪd wɔrmθ ənd gʊd ˈfɛloʊˌʃɪp
9,hi wɪl əˈlaʊ ə rɛr laɪ,hi wɪl əˈlaʊ ə rɛr laɪ
